In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
import category_encoders as ce
from sklearn.model_selection import GridSearchCV, KFold



In [2]:
df = pd.read_csv(R'E:\Mumbai Flat Real Estate Intelligence\datasets\mumbai_properties_post_feature_selection_v2.csv')

In [3]:
df.head()

,price,bedrooms,bathrooms,balcony,property_age,major_location,built_up_area,furnishing_type
0,0.17,1,1,2,New_property,Nalasopara West,550.0,Furnished
1,0.18,1,1,0,New_property,others,665.0,Furnished
2,0.18,1,1,2,New_property,Palghar,630.0,Furnished
3,0.18,1,1,2,New_property,Palghar,630.0,Unfurnished
4,0.18,1,2,1,New_property,Palghar,630.0,Furnished


In [4]:
df['furnishing_type'].value_counts()

furnishing_type
Furnished      5453
Unfurnished    4161
Name: count, dtype: int64

In [5]:
df.shape

(9614, 8)

In [6]:
df = df[df['major_location'] != 'Vakola']

In [7]:
df.shape

(9611, 8)

In [8]:
# df = df[df['price']<18]

In [9]:
# df.shape

In [10]:
X = df.drop(columns=['price'])
y = df['price']

In [11]:
# Applying the log1p transformation to the target variable
y_transformed = np.log1p(y)

### Ordinal Encoding


In [12]:
columns_to_encode = ['major_location',  'property_age', 'balcony','furnishing_type']


In [13]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedrooms', 'bathrooms', 'built_up_area']),
        ('cat', OrdinalEncoder(), columns_to_encode)
    ], 
    remainder='passthrough'
)

In [14]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [15]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [16]:

scores.mean(),scores.std()

(np.float64(0.7384415010872983), np.float64(0.013974063061669746))

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)


In [18]:
pipeline.fit(X_train,y_train)


,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [19]:
y_pred = pipeline.predict(X_test)

In [20]:
y_pred = np.expm1(y_pred)

In [21]:
mean_absolute_error(np.expm1(y_test),y_pred)

1.0760850631621757

In [22]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output

In [23]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [24]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))


In [25]:
model_output

[['linear_reg', np.float64(0.7384415010872983), 1.0760850631621757],
 ['svr', np.float64(0.7446173229027798), 1.0255749457664811],
 ['ridge', np.float64(0.7384417618897658), 1.076046094685717],
 ['LASSO', np.float64(0.018147290612791356), 1.907486662374778],
 ['decision tree', np.float64(0.8163529750867635), 0.777763129751163],
 ['random forest', np.float64(0.8965815913749928), 0.651841472720792],
 ['extra trees', np.float64(0.8562481926909742), 0.7425385062536007],
 ['gradient boosting', np.float64(0.8799930831369865), 0.7267691050287263],
 ['adaboost', np.float64(0.7356898907592615), 1.0647648726786139],
 ['mlp', np.float64(0.774728871682059), 0.9311601678328532],
 ['xgboost', np.float64(0.9236356664535151), 0.5670053339024663]]

In [26]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])


In [27]:
model_df.sort_values(['mae'])


,name,r2,mae
10,xgboost,0.923636,0.567005
5,random forest,0.896582,0.651841
7,gradient boosting,0.879993,0.726769
6,extra trees,0.856248,0.742539
4,decision tree,0.816353,0.777763
9,mlp,0.774729,0.931160
1,svr,0.744617,1.025575
8,adaboost,0.735690,1.064765
2,ridge,0.738442,1.076046
0,linear_reg,0.738442,1.076085


### OHE

In [34]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedrooms', 'bathrooms', 'built_up_area']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first'),['major_location','property_age','furnishing_type'])
    ], 
    remainder='passthrough'
)

In [35]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [36]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [37]:
scores.mean()

np.float64(0.9126758226051189)

In [38]:
scores.std()

np.float64(0.00556185924412332)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)


In [40]:
pipeline.fit(X_train,y_train)


,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [41]:
y_pred = pipeline.predict(X_test)



In [42]:
y_pred = np.expm1(y_pred)

In [43]:
mean_absolute_error(np.expm1(y_test),y_pred)

0.6368726471167551

In [44]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output

In [45]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [46]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

In [47]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [48]:
model_df.sort_values(['mae'])

,name,r2,mae
6,extra trees,0.916159,0.566067
9,mlp,0.919185,0.570736
10,xgboost,0.924092,0.574965
5,random forest,0.915026,0.579329
2,ridge,0.912170,0.636194
0,linear_reg,0.912676,0.636873
4,decision tree,0.860215,0.693673
7,gradient boosting,0.880794,0.713258
1,svr,0.785682,0.963674
8,adaboost,0.742037,1.026657


### OneHotEncoding With PCA

In [55]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedrooms', 'bathrooms', 'built_up_area']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False),['major_location','property_age'])
    ], 
    remainder='passthrough'
)

In [56]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('pca', PCA(n_components=0.95)),
    ('regressor', LinearRegression())
])

In [57]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [58]:
scores.mean()

np.float64(0.020503307020318308)

In [59]:

scores.std()

np.float64(0.008235909844130708)

In [60]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('pca', PCA(n_components=0.95)),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output
    

In [61]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [62]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

In [63]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [64]:
model_df.sort_values(['mae'])

,name,r2,mae
5,random forest,0.686651,1.079481
6,extra trees,0.651676,1.119145
4,decision tree,0.589933,1.228788
10,xgboost,0.613427,1.260955
7,gradient boosting,0.593137,1.307499
8,adaboost,0.249717,1.750911
1,svr,0.071194,1.824727
9,mlp,0.076866,1.882633
2,ridge,0.020503,1.905793
0,linear_reg,0.020503,1.905793


### Target Encoder

In [65]:
import category_encoders as ce

In [66]:


columns_to_encode = ['major_location', 'balcony', 'property_age', 'furnishing_type']

# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedrooms', 'bathrooms', 'built_up_area']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False),['property_age']),
        ('target_enc', ce.TargetEncoder(), ['major_location'])
    ], 
    remainder='passthrough'
)

In [67]:
!pip install category_encoders

In [68]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [69]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [70]:

scores.mean(),scores.std()

(np.float64(0.8907256650300992), np.float64(0.008092702807362219))

In [71]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output
    

In [72]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [73]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

In [74]:

model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])


In [75]:
model_df.sort_values(['mae'])


,name,r2,mae
10,xgboost,0.930499,0.539400
6,extra trees,0.926101,0.541827
5,random forest,0.928034,0.546414
7,gradient boosting,0.917957,0.592108
4,decision tree,0.877464,0.669979
9,mlp,0.893293,0.698419
0,linear_reg,0.890726,0.700720
2,ridge,0.890727,0.700744
1,svr,0.862140,0.793409
8,adaboost,0.856102,0.795600


### Hyperparameter Tuning

In [76]:
from sklearn.model_selection import GridSearchCV


In [77]:
df.head()

,price,bedrooms,bathrooms,balcony,property_age,major_location,built_up_area,furnishing_type
0,0.17,1,1,2,New_property,Nalasopara West,550.0,Furnished
1,0.18,1,1,0,New_property,others,665.0,Furnished
2,0.18,1,1,2,New_property,Palghar,630.0,Furnished
3,0.18,1,1,2,New_property,Palghar,630.0,Unfurnished
4,0.18,1,2,1,New_property,Palghar,630.0,Furnished


In [84]:
param_grid = {
    'regressor__n_estimators': [50, 100, 150],
    'regressor__max_depth': [None, 10, 20, 25, 30],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 4],
    'regressor__max_features': ['auto', 'sqrt'],
    'regressor__max_samples': [0.5, 0.75, 1.0]
}


In [85]:
columns_to_encode = [ 'balcony', 'furnishing_type']

# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedrooms', 'bathrooms', 'built_up_area']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False),['property_age']),
        ('target_enc', ce.TargetEncoder(), ['major_location'])
    ], 
    remainder='passthrough'
)

In [86]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])

In [87]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)


In [88]:
search = GridSearchCV(pipeline, param_grid, cv=kfold, scoring='r2', n_jobs=-1, verbose=4)

In [89]:
search.fit(X, y_transformed)

Fitting 10 folds for each of 810 candidates, totalling 8100 fits


e:\Mumbai Flat Real Estate Intelligence\mumbai_flat_env\Lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
4050 fits failed out of a total of 8100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1685 fits failed with the following error:
Traceback (most recent call last):
  File "e:\Mumbai Flat Real Estate Intelligence\mumbai_flat_env\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "e:\Mumbai Flat Real Estate Intelligence\mumbai_flat_env\Lib\site-packages\sklearn\base.py", line 1365, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "e:\Mumbai Fl

,estimator,Pipeline(step...Regressor())])
,param_grid,"{'regressor__max_depth': [None, 10, ...], 'regressor__max_features': ['auto', 'sqrt'], 'regressor__max_samples': [0.5, 0.75, ...], 'regressor__min_samples_leaf': [1, 2, ...], ...}"
,scoring,'r2'
,n_jobs,-1
,refit,True
,cv,KFold(n_split... shuffle=True)
,verbose,4
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('num', ...), ('cat', ...), ...]"


In [90]:
final_pipe = search.best_estimator_

In [91]:
search.best_params_


{'regressor__max_depth': None,
 'regressor__max_features': 'sqrt',
 'regressor__max_samples': 1.0,
 'regressor__min_samples_leaf': 1,
 'regressor__min_samples_split': 5,
 'regressor__n_estimators': 150}

In [92]:
search.best_score_


np.float64(0.9255822117592138)

In [93]:
final_pipe.fit(X,y_transformed)


,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [94]:
y_pred = final_pipe.predict(X_test)


In [95]:
y_pred = np.expm1(y_pred)


In [96]:
mean_absolute_error(np.expm1(y_test),y_pred)

0.33176703606975905

best selected model

In [97]:
numeric_cols = ['bedrooms', 'bathrooms', 'built_up_area']
onehot_cols = ['balcony', 'property_age', 'furnishing_|type']
target_cols = ['major_location']

In [98]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),
        ('cat_onehot', OneHotEncoder(drop='first', sparse_output=False), onehot_cols),
        ('target', ce.TargetEncoder(), target_cols)
    ],
    remainder='drop'
)

In [99]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])

In [ ]:
param_grid = {
    'regressor__n_estimators': [100],
    'regressor__max_depth': [None, 10, 20, 25, 30],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 4],
    'regressor__max_features': ['auto', 'sqrt'],
    'regressor__max_samples': [0.5, 0.75, 1.0]
}

In [104]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

search = GridSearchCV(
    pipeline,
    param_grid,
    cv=kfold,
    scoring='r2',
    n_jobs=-1,
    verbose=4
)

In [106]:
# search.fit(X, y_transformed)

In [ ]:
final_pipe = search.best_estimator_

In [ ]:
y_pred = final_pipe.predict(X_test)

In [ ]:
y_pred = np.expm1(y_pred)

In [ ]:
mae = mean_absolute_error(np.expm1(y_test), y_pred)

In [ ]:
print("Best Params:", search.best_params_)
print("Best R2 Score:", search.best_score_)
print("MAE:", mean_absolute_error(np.expm1(y_test), y_pred))

Best Params: {'regressor__max_depth': 20, 'regressor__max_features': 'sqrt', 'regressor__max_samples': 1.0, 'regressor__min_samples_leaf': 1, 'regressor__min_samples_split': 2, 'regressor__n_estimators': 300}
Best R2 Score: 0.9227966512910777
MAE: 0.243559218444499


### exporting pkl 

In [ ]:
import pickle

with open("pipeline.pkl", "wb") as file:
    pickle.dump(final_pipe, file)

In [ ]:
with open('df.pkl', 'wb') as file:
    pickle.dump(X, file)

In [ ]:

X

,bedrooms,bathrooms,balcony,property_age,major_location,built_up_area,furnishing_type
0,1,1,2,New_property,Nalasopara West,550.0,Furnished
1,1,1,0,New_property,others,665.0,Furnished
2,1,1,2,New_property,Palghar,630.0,Furnished
3,1,1,2,New_property,Palghar,630.0,Unfurnished
4,1,2,1,New_property,Palghar,630.0,Furnished
...,...,...,...,...,...,...,...
9609,5,5,0,Old,Malabar Hill,3699.0,Unfurnished
9610,5,5,0,Mid Age,Malabar Hill,3181.0,Furnished
9611,5,6,1,Old,Malabar Hill,2850.0,Furnished
9612,6,4,3,Mid Age,Lower Parel,5042.0,Furnished


In [ ]:
X.shape

(9611, 7)

In [ ]:
df.to_csv('mumbai_properties_final.csv', index=False)

### Trying out the predictions

In [ ]:
X.columns

Index(['bedrooms', 'bathrooms', 'balcony', 'property_age', 'major_location',
       'built_up_area', 'furnishing_type'],
      dtype='object')

In [ ]:
X.iloc[0].values

array([np.int64(1), np.int64(1), '2', 'New_property', 'Nalasopara West',
       np.float64(550.0), 'Furnished'], dtype=object)

In [ ]:
data = [[ 2, 2, '0', 'Mid Age', 'Mulund West', 650,  'Unfurnished']]
columns = ['bedrooms', 'bathrooms', 'balcony', 'property_age', 'major_location',
       'built_up_area',  'furnishing_type']

# Convert to DataFrame
one_df = pd.DataFrame(data, columns=columns)

one_df

,bedrooms,bathrooms,balcony,property_age,major_location,built_up_area,furnishing_type
0,2,2,0,Mid Age,Mulund West,650,Unfurnished


In [ ]:
np.expm1(final_pipe.predict(one_df))

array([1.76952293])

In [ ]:
X.dtypes

bedrooms             int64
bathrooms            int64
balcony             object
property_age        object
major_location      object
built_up_area      float64
furnishing_type     object
dtype: object

In [ ]:
sorted(X['major_location'].unique().tolist())


['Agripada',
 'Andheri East',
 'Andheri West',
 'Bandra East',
 'Bandra West',
 'Bhandup East',
 'Bhandup West',
 'Bhayandar East',
 'Bhayandar West',
 'Bhoiwada',
 'Boisar',
 'Bolinj',
 'Borivali East',
 'Borivali West',
 'Byculla',
 'Chakala',
 'Chembur',
 'Chikhal Dongari',
 'Churchgate',
 'Colaba',
 'Cuffe Parade',
 'Cumballa Hill',
 'Dadar East',
 'Dadar West',
 'Dahisar',
 'Dahisar East',
 'Dahisar West',
 'Dattapada',
 'Deonar',
 'Evershine Nagar',
 'Gandhi Nagar',
 'Ghatkopar East',
 'Ghatkopar West',
 'Girgaon',
 'Goregaon',
 'Goregaon East',
 'Goregaon West',
 'Govandi',
 'Hindu Colony',
 'Hmpl Surya Nagar',
 'Jogeshwari East',
 'Jogeshwari West',
 'Juhu',
 'Juhu Scheme',
 'Jvpd Scheme',
 'Kala Nagar',
 'Kamathipura',
 'Kanchpada',
 'Kandivali East',
 'Kandivali West',
 'Kanjurmarg East',
 'Kanjurmarg West',
 'Kashimira',
 'Khar West',
 'Kherwadi',
 'Kurla East',
 'Kurla West',
 'Lower Parel',
 'Lower Parel East',
 'Lower Parel West',
 'Madanpura',
 'Madh',
 'Mahalakshmi',
 '